# 데이터 로딩, 저장, 파일 형식

## 텍스트 파일

- pandas 파일 파싱 함수
- read_csv: 파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어온다.
- read_table: 파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어온다. 데이터 구분자는 탭('\t')을 기본으로 한다.
- read_fwf: 고정폭 칼럼 형식에서 데이터를 읽어온다.
- read_clipboard: 클립보드에 있는 데이터를 읽어오는 read_table 함수. 웹페이지에서 표를 긁어올 때 유용하다.

- 텍스트 데이터를 DataFrame으로 읽어오는 함수들을 다음과 같은 몇 가지 옵션을 사용한다.
- **색인**: 반환하는 DataFrame에서 하나 이상의 칼럼을 색인으로 지정할 수 있다. 지정하지 않아도 된다.
- **자료형 추론과 데이터 변환**: 사용자가 정의 값 변환과 비어있는 값을 위한 사용자 리스트를 포함한다.
- **날짜 분석**: 여러 칼럼에 걸쳐 있는 날짜와 시간 정보를 하나의 칼럼에 조합해서 결과에 반영한다.
- **반복**: 여러 파일에 걸쳐 있는 자료를 반복적으로 읽어올 수 있다.
- **정제되지 않은 데이터 처리**: 로우나 꼬리말, 주석 건너뛰기 또는 천 단위마다 쉼표로 구분된 숫자 같은 일을 처리한다.

- 자료형 추론은 매우 종요한 기능이다.
- 날짜나 다른 몇 가지 사용자 자료형을 처리하려면 어떤 칼럼이 숫자인지 불리언인지 문자열인지 지정해줄 필요가 없다.

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series

In [2]:
!cat ../data/ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [3]:
df = pd.read_csv('../data/ex1.csv')

In [4]:
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- read_table에 구분자를 쉼표로 지정해서 읽어올 수도 있다.

In [5]:
pd.read_table('../data/ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [6]:
!cat ../data/ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


- 칼럼의 이름이 없는 파일도 있다.
- 이런 파일을 처리하려면 pandas가 자동으로 칼럼 이름을 생성하도록 하거나 직접 칼럼 이름을 지정할 수도 있다.

In [7]:
pd.read_csv('../data/ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [8]:
pd.read_csv('../data/ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [14]:
# pd.read_csv('../data/ex2.csv', columns=['a', 'b', 'c', 'd', 'message']) # 에러 발생

- message 칼럼을 색인으로 하는 DataFrame을 반환하려면 index_col 인자에 message라는 칼럼을 지정해서 색인으로 만들 수 있다.
* message라는 항목은 names 리스트에 반드시 포함되어 있어야 한다.

In [20]:
names = ['a', 'b', 'c', 'd', 'message']

In [21]:
pd.read_csv('../data/ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


- 계층적 색인을 지정하고 싶다면 칼럼 번호나 이름의 리스트를 넘기면 된다.

In [12]:
!cat ../data/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('../data/csv_mindex.csv', index_col=['key1', 'key2'])

In [14]:
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

- 고정된 구분자 없이 공백이나 다른 패턴으로 필드를 구분해 놓은 경우 read_table의 구분자로 정규표현식을 사용하면 된다.

In [15]:
list(open('../data/ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

- 공백문자를 표현할 수 있는 정규표현식 \s+ 를 사용하여 처리한다.

In [16]:
result = pd.read_table('../data/ex3.txt', sep='\s+')

In [17]:
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


- 첫 번째 로우는 다른 로우보다 컬럼이 하나 적기 때문에 read_table은 첫 번째 칼럼이 DataFrame의 색인이 되어야 한다고 추론한다.

In [18]:
!cat ../data/ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [19]:
pd.read_csv('../data/ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


- 누락된 값을 잘 처리하는 일은 파일을 읽는 과정에서 자주 발생하는 일이고 중요한 문제다.
- 텍스트 파일에서 누락된 값은 표기되지 않거나 구분하기 쉬운 특수한 문자로 표기된다.

In [20]:
!cat ../data/ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [21]:
result = pd.read_csv('../data/ex5.csv')

In [22]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [23]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


- na_values 옵션은 리스트나 문자열 집합을 받아서 누락된 값을 처리한다.

In [24]:
result = pd.read_csv('../data/ex5.csv', na_values=['NULL'])

In [25]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


- 열마다 다른 NA 문자를 사전 값으로 넘겨 처리할 수도 있다.

In [26]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}

In [27]:
pd.read_csv('../data/ex5.csv',  na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 텍스트 파일 조금씩 읽어오기

- 매우 큰 파일을 처리할 때 인자를 제대로 주었는지 알아보기 위해 파일의 일부분만 읽어보거나 여러 파일 중에서 몇 개의 파일만 읽어서 확인해본다.
- 파일 전체를 읽는 대신 처음 몇 줄만 읽어보고 싶다면 nrows 옵션을 사용한다.

In [28]:
pd.read_csv('../data/ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


- 파일을 여러 조각에 나누어서 읽고 싶다면 chunksize 옵션으로 로우의 갯수를 주면 된다.

In [28]:
chunker = pd.read_csv('../data/ex6.csv', chunksize=1000)

In [29]:
chunker

In [30]:
#chunker = pd.read_csv('../data/ex6.csv', chunksize=1000)
tot = pd.Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)
tot = tot.sort_values(ascending=False)

In [31]:
tot[:10]

E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

## 데이터를 텍스트 형식으로 기록하기

In [32]:
data = pd.read_csv('../data/ex5.csv')

In [33]:
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [34]:
data.to_csv('../data/out.csv')

In [35]:
!cat ../data/out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


- 다른 구분자도 사용 가능하다.
- 콘솔에서 확인할 수 있도록 sys.stdout에 결과를 출력할 수도 있다.

In [36]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


- 결과에서 누락된 값은 비어있는 문자열로 출력된다.
- 원하는 값으로 지정할 수 있다.

In [37]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


- 다른 옵션이 명시되지 않으면 로우와 칼럼의 이름도 함께 기록된다.
- 로우와 칼럼의 이름을 포함하지 않으려면 다음과 같이 처리한다.

In [38]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


- 컬럼의 일부분만 기록할 수도 있고, 순서를 직접 지정할 수도 있다.

In [39]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


- Series에도 to_csv 메서드가 있다.

In [40]:
dates = pd.date_range('1/1/2000', periods=7)

In [41]:
ts = pd.Series(np.arange(7), index=dates)

In [42]:
ts.to_csv(sys.stdout)

2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


## 수동으로 구분 형식 처리하기

In [34]:
!cat ../data/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


- 구분자가 한 글자인 파일은 파이썬 내장 csv 모듈을 이용해서 처리할 수 있다.

In [37]:
import csv
f = open('../data/ex7.csv')
reader = csv.reader(f)

In [38]:
for line in reader:
    print(line)
    print(type(line))

['a', 'b', 'c']
<class 'list'>
['1', '2', '3']
<class 'list'>
['1', '2', '3', '4']
<class 'list'>


In [46]:
lines = list(csv.reader(open('../data/ex7.csv')))

In [47]:
header, values = lines[0], lines[1:]

In [48]:
values

[['1', '2', '3'], ['1', '2', '3', '4']]

In [49]:
list(zip(*values))

[('1', '1'), ('2', '2'), ('3', '3')]

In [50]:
list(zip(*values))

[('1', '1'), ('2', '2'), ('3', '3')]

In [51]:
data_dict = {h: v for h, v in zip(header, zip(*values))}

In [52]:
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

- csv 파일은 다양한 형태로 존재한다.
- 다양한 구분자, 문자열을 둘러싸는 방법, 개행문자 같은 것은 csv.Dialect를 상속받아 새로운 클래스를 정의해서 처리한다.
* Dialect: 방언

In [53]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'
    quoting = csv.QUOTE_MINIMAL

In [54]:
f = open('../data/ex8.csv')
reader = csv.reader(f, dialect=my_dialect)

In [55]:
for line in reader:
    print(line)

['1', '2', '3']


- 구분자가 한 글자를 초과하는 고정 길이를 가진다면 csv 모듈을 사용할 수 없다.
- 이런 경우에는 줄을 나누고 문자열의 split 메서드나 정규표현식의 메서드인 re.split 등을 이용하여 가공해야 한다.

In [56]:
with open('../data/mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## JSON 데이터

In [57]:
obj = """
      { "name":  "Wes",
        "places_lived": ["United States", "Spain", "Germany"],
        "pet": null,
        "siblings": [{"name": "Scott", "age": 25, "pets": "Zuko"},
                     {"name": "Katie", "age": 38, "pets": "Cisco"}]
      }
      """

- JSON은 널 값인 null과 몇 가지 사소한 주의사항(리스트의 마지막에 쉼표가 있으면 안된다.)을 제외하면 파이썬 코드와 유사하다.
- 기본 자료형은 객체(사전), 배열(리스트), 문자열, 숫자, 불리언 그리고 널이다.
- 객체의 키는 반드시 문자열이어야 한다.

- JSON 문자열을 파이썬 형태로 변환하기 위해서는 json.loads를 사용한다.

In [58]:
import json
result = json.loads(obj)

In [59]:
result

{'name': 'Wes',
 'pet': None,
 'places_lived': ['United States', 'Spain', 'Germany'],
 'siblings': [{'age': 25, 'name': 'Scott', 'pets': 'Zuko'},
  {'age': 38, 'name': 'Katie', 'pets': 'Cisco'}]}

In [60]:
asjson = json.dumps(result)

In [61]:
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 25, "pets": "Zuko"}, {"name": "Katie", "age": 38, "pets": "Cisco"}]}'

- JSON 객체의 리스트를 DataFrame 생성자로 넘기고 데이터 필드를 선택할 수 있다.

In [62]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])

In [63]:
siblings

,name,age
0,Scott,25
1,Katie,38


## 엑셀 파일 데이터 읽기/저장하기

In [64]:
frame = pd.read_excel('../data/ex1.xlsx')

In [65]:
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [66]:
import xlsxwriter
writer = pd.ExcelWriter('../data/ex1_out.xlsx', engine='xlsxwriter')
frame.to_excel(writer, 'Sheet1')

In [67]:
workbook  = writer.book
worksheet = writer.sheets['Sheet1']

In [68]:
writer.save()

## 데이터베이스와 함께 사용하기

In [69]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL, d INTEGER);
"""
con = sqlite3.connect(':memory:')
con.execute(query)
con.commit()

In [70]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

In [71]:
cursor = con.execute('select * from test')

In [72]:
rows = cursor.fetchall()

In [73]:
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

In [74]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [75]:
frame = DataFrame(rows, columns=list(zip(*cursor.description))[0])

In [76]:
frame

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


In [77]:
frame1 = pd.read_sql('select * from test', con)

In [78]:
frame1

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
